In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163008 sha256=8f68a4ff86c559c526241b980bb591bb8fb448fba91250c28096a95561aec388
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import  train_test_split
from surprise import accuracy

In [ ]:
# Install Data Set
tag_df = pd.read_csv("/content/drive/MyDrive/movieRecomSystData/tags.csv")
movies_df = pd.read_csv("/content/drive/MyDrive/movieRecomSystData/movies.csv")
ratings_df = pd.read_csv("/content/drive/MyDrive/movieRecomSystData/ratings.csv")

small_ratings = ratings_df.head(70000)

In [ ]:
# Take inputs from user
user_tags = ['funny', 'family', 'animation']
user_movies = [3114, 60756, 1569]

In [ ]:
# Kullanıcının girdilerini işle
#user_tags_list = user_tags.split(',')
#user_movies_list = [int(movie_id) for movie_id in user_movies.split(',')]


In [ ]:
# Combine the user's favorite movies with matching films
selected_movies = movies_df[movies_df['movieId'].isin(user_movies)]

In [ ]:
matching_movies = pd.DataFrame(columns=movies_df.columns)
for tag in user_tags:
    matching_movies = pd.concat([matching_movies, movies_df[movies_df['genres'].str.contains(tag, case=False)]], ignore_index=True)

In [ ]:
# Kullanıcının sevdiği filmleri ve eşleşen filmleri birleştir
user_input_movies = pd.concat([selected_movies, matching_movies], ignore_index=True).drop_duplicates()

In [ ]:
# Kullanıcının girdilerine göre bir veri seti oluştur
user_data = {'userId': [999] * len(user_input_movies),
             'movieId': user_input_movies['movieId'].tolist(),
             'rating': [5.0] * len(user_input_movies)}
user_df = pd.DataFrame(user_data)

In [ ]:
# Veri setine kullanıcı girdilerini ekleyin
new_ratings_df = pd.concat([small_ratings, user_df], ignore_index=True)

In [ ]:
# Surprise kütüphanesi için gerekli işlemleri yapın
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(new_ratings_df[['userId', 'movieId', 'rating']], reader)
trainset = dataset.build_full_trainset()

In [ ]:
# KNN modelini eğitin
sim_options = {'name': 'cosine', 'user_based': False}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Kullanıcının için film önerileri yapın
user_recommendations = model.get_neighbors(999, k=50)

In [ ]:
# Önerilen filmleri gösterin
print("Film Önerileri:")
print(movies_df[movies_df['movieId'].isin(user_recommendations)][['movieId', 'title']])

Film Önerileri:
     movieId                                              title
5          6                                        Heat (1995)
7          8                                Tom and Huck (1995)
14        15                            Cutthroat Island (1995)
29        30  Shanghai Triad (Yao a yao yao dao waipo qiao) ...
46        47                        Seven (a.k.a. Se7en) (1995)
52        53                                    Lamerica (1994)
57        58                  Postman, The (Postino, Il) (1994)
64        65                                    Bio-Dome (1996)
67        68                 French Twist (Gazon maudit) (1995)
69        70                         From Dusk Till Dawn (1996)
71        72                       Kicking and Screaming (1995)
72        73                             Misérables, Les (1995)
87        88                                 Black Sheep (1996)
93        95                                Broken Arrow (1996)
100      102            